# 👤 **insightface**
<hr>
Original Github repo <a href="https://github.com/deepinsight/insightface">here</a>. Adapted for <a href="https://mmla.gse.harvard.edu/">ez-mmla</a>, <a href="http://lit.gse.harvard.edu/">Harvard LITLab</a> (Schneider et al.)
<br><br>
<!-- Ideally, put a GIF or an image here that best portrays the demo or the model. -->
<center> <img width="300px" src="https://drive.google.com/uc?id=1VAcjh7lX79_p46YKXlGTfZsAghITNEUL">
</center>
<center>The results obtained from insightface gender & age recognition </center>


> an open source 2D&3D deep face analysis toolbox, mainly based on PyTorch and MXNet

This
**Some notes:**
* Need PyTorch 1.6+ and/or MXNet=1.6-1.8, with Python 3.x.
* This noteboook contains demos from gender & age recognition and face swapping.


## 🔧 **Setup**
<hr>
⏳ <b>Colab/Colab Pro</b>: 5 minutes


In [ ]:
#@title #### **Install and import dependencies**
#@markdown Install the alphapose Python library and necessary dependencies.
from IPython.display import display, HTML, clear_output

completed_actions = []
current_action = ""

def start_step(action):
  global current_action
  current_action = action
  display(HTML(f"<b style='font-size: 15px'>{action} ⏳</b>"))
  display(HTML("<hr>"))

def finished():
  global current_action, completed_actions
  current_action = ""
  display(HTML(f"<b style='font-size: 15px; color: green;'>Done ✅ </b>"))
  completed_actions = []

def end_step():
  global current_action, completed_actions
  completed_actions.append(current_action)
  current_action = ""
  clear_output()
  for action in completed_actions:
    display(HTML(f"<b style='font-size: 15px'>{action} ✅ </b>"))
    display(HTML("<hr>"))

start_step("Install insightface")
!pip install -U insightface
end_step()

start_step("Install onnxruntime")
!pip install onnxruntime --q
end_step()


start_step("Install pytube")
!pip install pytube --q
end_step()

start_step("Install imageio")
!pip install imageio==2.4.1 --q
end_step()

start_step("Import necessary libraries")
from IPython.display import display, Javascript,HTML, Image
from google.colab.output import eval_js
from base64 import b64decode,b64encode
from google.colab import files
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
from scipy.io.wavfile import write
import os
from google.colab import drive
from pytube import YouTube
from moviepy.editor import *
import cv2
import torch
import yaml, scipy, os
import numpy as np
import insightface
from insightface.app import FaceAnalysis
end_step()

start_step("Downloading models")
!gdown https://drive.google.com/u/1/uc?id=1GW7Q41Uk4H30wVFL2Tl4Kl8MWIV4fVFC&export=download&confirm=t&uuid=0f573fda-94ec-468a-93bb-31db5b59da59&at=ANzk5s4fqBWY9icjAlD86Q3HJ--k:1680463073493
end_step()

finished()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 82.3 MB/s eta 0:00:00


## 🕹 **Demo - Age & Gender Recognition**
<hr>

In [ ]:
#@title #### **Image input**

import ipywidgets as widgets
from google.colab import files
from IPython.display import Video, Image
from pytube import YouTube
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode,b64encode
import cv2

input_file = None


# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    clear_output()
    output = widgets.Output()
    dropdown = widgets.Dropdown(options = options.keys(), description='Actions:')

    def on_option_change(choice):
        global input_file
        clear_output()
        # Run the associated function
        input_file = options[choice]()
        clear_output()
        display(input_widgets)
        on_finished(input_file)

    dropdown.observe(lambda e: on_option_change(e.new), names='value')
    input_widgets = widgets.HBox([dropdown])
    display(input_widgets)

    # Initially run the first option (the demo)
    on_option_change(list(options.keys())[0])

# This function runs whenever the action user chose finishes.
def on_finished(fn):
  global image_or_video
  if fn:
    display(HTML(f"<hr><b>Below is a preview of the file:</b> <code>{fn}</code><br><br>"))

    display(Image(fn, embed=True, width=400))

# Functions corresponding to the actions the user selected!
def demo():
  global image_or_video
  image_or_video = "image"

  youtube = YouTube('https://youtube.com/shorts/lH7wI6HW4-4')
  stream=youtube.streams.get_highest_resolution()
  videofile=stream.download()
  vidcap = cv2.VideoCapture(videofile)
  success,image = vidcap.read()
  file="/content/image.jpg"
  cv2.imwrite(file, image)
  return file

def take_photo(filename='photo.jpg', quality=0.8):
  global image_or_video
  image_or_video = "image"

  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def drive_file_chooser(exts):
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  files = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the files obtained from your Google Drive. </p>
  <b> Choose a file from the list below. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for filename in filenames:
      # change the extensions to the type of file we are looking for
      if filename.endswith(tuple(exts)):
          FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, filename)
          i += 1
          files.append(os.path.join(root,filename))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return files[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch files from your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser(["png", "jpg", "mp4", "avi", "mov"])

def upload():
  try:
    uploaded = files.upload()
    fn = list(uploaded.keys())[0]
    return "/content/" + fn
  except:
    return None

dropdown_menu({
    "Demo picture": demo,
    "Take a picture": take_photo,
    "Upload file": upload,
    "Upload from Google Drive": google_drive,
}, on_finished)


In [ ]:
#@title #### **Result output** { run: "auto" }
#@markdown Specify how you want to save your results and run this cell.
import time
result_output = None
output = "Download results" #@param ["Don't save results", "Download results", "Save to Google Drive"]

# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    global result_output
    clear_output()
    # Run the associated function
    result_output = options[output]()
    clear_output()
    on_finished(result_output)

def on_finished(input):
  clear_output()
  global result_output
  if input:
    result_output = input
  if not input:
    return
  elif input == "download":
    display(HTML(f"<hr><b>Results will be downloaded when the demo is finished running.</b><br><br>"))
  else:
    display(HTML(f"<hr><b>Results will be stored to </b><code>{input}</code><b> when the demo is finished running.</b><br><br>"))

def drive_file_chooser():
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  dirs = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the folders obtained from your Google Drive. </p>
  <b> Choose a folder to store the model results. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for directory in directories:
      # change the extensions to the type of file we are looking for
        FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, directory)
        i += 1
        dirs.append(os.path.join(root,directory))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return dirs[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser()

def nothing():
  return None

def download():
  return "download"

dropdown_menu({
    "Don't save results": nothing,
    # "Save to Google Drive": google_drive,
    "Download results": download,
}, on_finished)


In [ ]:
#@title #### **Run the demo and save results**
#@markdown Run this cell to run the model. <br>


from google.colab import files

app = FaceAnalysis(providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))
# img = ins_get_image('t1')
img = cv2.imread(input_file)
faces = app.get(img)
rimg = app.draw_on(img, faces)
cv2.imwrite("./test_output.jpg", rimg)
display(Image('test_output.jpg'))

import json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open("result.json", "w") as file:
  count=0
  for face in faces:
    count+=1
    jsonString = json.dumps({count:face}, cls=NpEncoder)
    file.write(jsonString)
  file.close()

# Handle output choice
if result_output == "download":
  files.download('test_output.jpg')
  files.download('result.json')

# elif result_output:
#   if image_or_video == 'image':
#       image_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))
#   else:
#       video_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))



## 🕹 **Demo - Face swapping**
<hr>

In [ ]:
#@title #### **Image input**

import ipywidgets as widgets
from google.colab import files
from IPython.display import Video, Image
from pytube import YouTube
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode,b64encode
import cv2

input_file = None


# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    clear_output()
    output = widgets.Output()
    dropdown = widgets.Dropdown(options = options.keys(), description='Actions:')

    def on_option_change(choice):
        global input_file
        clear_output()
        # Run the associated function
        input_file = options[choice]()
        clear_output()
        display(input_widgets)
        on_finished(input_file)

    dropdown.observe(lambda e: on_option_change(e.new), names='value')
    input_widgets = widgets.HBox([dropdown])
    display(input_widgets)

    # Initially run the first option (the demo)
    on_option_change(list(options.keys())[0])

# This function runs whenever the action user chose finishes.
def on_finished(fn):
  global image_or_video
  if fn:
    display(HTML(f"<hr><b>Below is a preview of the file:</b> <code>{fn}</code><br><br>"))

    display(Image(fn, embed=True, width=400))

# Functions corresponding to the actions the user selected!
def demo():
  global image_or_video
  image_or_video = "image"

  youtube = YouTube('https://youtu.be/vzAUdqPy4lw')
  stream=youtube.streams.get_highest_resolution()
  videofile=stream.download()
  vidcap = cv2.VideoCapture(videofile)
  success,image = vidcap.read()
  file="/content/image.jpg"
  cv2.imwrite(file, image)
  return file

def take_photo(filename='photo.jpg', quality=0.8):
  global image_or_video
  image_or_video = "image"

  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def drive_file_chooser(exts):
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  files = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the files obtained from your Google Drive. </p>
  <b> Choose a file from the list below. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for filename in filenames:
      # change the extensions to the type of file we are looking for
      if filename.endswith(tuple(exts)):
          FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, filename)
          i += 1
          files.append(os.path.join(root,filename))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return files[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch files from your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser(["png", "jpg", "mp4", "avi", "mov"])

def upload():
  try:
    uploaded = files.upload()
    fn = list(uploaded.keys())[0]
    return "/content/" + fn
  except:
    return None

dropdown_menu({
    "Demo picture": demo,
    "Take a picture": take_photo,
    "Upload file": upload,
    "Upload from Google Drive": google_drive,
}, on_finished)


In [ ]:
#@title #### **Result output** { run: "auto" }
#@markdown Specify how you want to save your results and run this cell.
import time
result_output = None
output = "Download results" #@param ["Don't save results", "Download results", "Save to Google Drive"]

# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    global result_output
    clear_output()
    # Run the associated function
    result_output = options[output]()
    clear_output()
    on_finished(result_output)

def on_finished(input):
  clear_output()
  global result_output
  if input:
    result_output = input
  if not input:
    return
  elif input == "download":
    display(HTML(f"<hr><b>Results will be downloaded when the demo is finished running.</b><br><br>"))
  else:
    display(HTML(f"<hr><b>Results will be stored to </b><code>{input}</code><b> when the demo is finished running.</b><br><br>"))

def drive_file_chooser():
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  dirs = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the folders obtained from your Google Drive. </p>
  <b> Choose a folder to store the model results. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for directory in directories:
      # change the extensions to the type of file we are looking for
        FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, directory)
        i += 1
        dirs.append(os.path.join(root,directory))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return dirs[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser()

def nothing():
  return None

def download():
  return "download"

dropdown_menu({
    "Don't save results": nothing,
    # "Save to Google Drive": google_drive,
    "Download results": download,
}, on_finished)


In [ ]:
#@title #### **Run the demo and save results**
#@markdown Specify the source face and the faces you hope to change. The faces will labeled by their position (0, 1, ..., face_number0-1 from left to right). <br>
face_source = '2' #@param {type:"string"}
face_change ='0,1,3,4,5' #@param {type:"string"}

from insightface.data import get_image as ins_get_image
from google.colab import files
from google.colab.patches import cv2_imshow

face_source = int(face_source)
face_change = face_change.split(',')
face_change = [int(a) for a in face_change]

app = FaceAnalysis(name='buffalo_l')
start_step("Initializing models")
!mv inswapper_128.onnx ../root/.insightface/models
end_step()
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('inswapper_128.onnx', download=True, download_zip=True)
img = cv2.imread(input_file)
display(HTML("<b>Original Image Below</b>"))
cv2_imshow(img)

faces = app.get(img)
faces = sorted(faces, key = lambda x : x.bbox[0])
res = img.copy()

source_face = faces[face_source]

for i in face_change:
    face = faces[i]
    res = swapper.get(res, face, source_face, paste_back=True)

cv2.imwrite("./swapped.jpg", res)

res = []
for i in face_change:
    face = faces[i]
    _img, _ = swapper.get(img, face, source_face, paste_back=False)
    res.append(_img)
res = np.concatenate(res, axis=1)
cv2.imwrite("./swapped2.jpg", res)


display(Image('swapped.jpg'))
display(Image('swapped2.jpg'))


# Handle output choice
if result_output == "download":
  files.download('swapped.jpg')
  files.download('swapped2.jpg')

# elif result_output:
#   if image_or_video == 'image':
#       image_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))
#   else:
#       video_prediction.to_csv(os.path.join(result_output, 'pyfeat_result.csv'))



## 🕹 **Demo - Face blurring**
<hr>

In [ ]:
#@title #### **Image input**

import ipywidgets as widgets
from google.colab import files
from IPython.display import Video, Image
from pytube import YouTube
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode,b64encode
import cv2

input_file = None


# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    clear_output()
    output = widgets.Output()
    dropdown = widgets.Dropdown(options = options.keys(), description='Actions:')

    def on_option_change(choice):
        global input_file
        clear_output()
        # Run the associated function
        input_file = options[choice]()
        clear_output()
        display(input_widgets)
        on_finished(input_file)

    dropdown.observe(lambda e: on_option_change(e.new), names='value')
    input_widgets = widgets.HBox([dropdown])
    display(input_widgets)

    # Initially run the first option (the demo)
    on_option_change(list(options.keys())[0])

# This function runs whenever the action user chose finishes.
def on_finished(fn):
  global image_or_video
  if fn:
    display(HTML(f"<hr><b>Below is a preview of the file:</b> <code>{fn}</code><br><br>"))

    display(Image(fn, embed=True, width=400))

# Functions corresponding to the actions the user selected!
def demo():
  global image_or_video
  image_or_video = "image"

  youtube = YouTube('https://youtu.be/vzAUdqPy4lw')
  stream=youtube.streams.get_highest_resolution()
  videofile=stream.download()
  vidcap = cv2.VideoCapture(videofile)
  success,image = vidcap.read()
  file="/content/image.jpg"
  cv2.imwrite(file, image)
  return file

def take_photo(filename='photo.jpg', quality=0.8):
  global image_or_video
  image_or_video = "image"

  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def drive_file_chooser(exts):
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  files = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the files obtained from your Google Drive. </p>
  <b> Choose a file from the list below. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for filename in filenames:
      # change the extensions to the type of file we are looking for
      if filename.endswith(tuple(exts)):
          FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, filename)
          i += 1
          files.append(os.path.join(root,filename))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return files[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch files from your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser(["png", "jpg", "mp4", "avi", "mov"])

def upload():
  try:
    uploaded = files.upload()
    fn = list(uploaded.keys())[0]
    return "/content/" + fn
  except:
    return None

dropdown_menu({
    "Demo picture": demo,
    "Take a picture": take_photo,
    "Upload file": upload,
    "Upload from Google Drive": google_drive,
}, on_finished)


In [ ]:
#@title #### **Result output** { run: "auto" }
#@markdown Specify how you want to save your results and run this cell.
import time
result_output = None
output = "Don't save results" #@param ["Don't save results", "Download results", "Save to Google Drive"]

# The function takes in a dictionary of options and their corresponding functions
def dropdown_menu(options, on_finished):
    global result_output
    clear_output()
    # Run the associated function
    result_output = options[output]()
    clear_output()
    on_finished(result_output)

def on_finished(input):
  clear_output()
  global result_output
  if input:
    result_output = input
  if not input:
    return
  elif input == "download":
    display(HTML(f"<hr><b>Results will be downloaded when the demo is finished running.</b><br><br>"))
  else:
    display(HTML(f"<hr><b>Results will be stored to </b><code>{input}</code><b> when the demo is finished running.</b><br><br>"))

def drive_file_chooser():
  # This function renders a file chooser which finds all the files of valid extensions
  # inside your Google Drive and allows the user to select the file
  dirs = []
  FILE_HTML = ""

  JS = """
  <style>
  .list-item {
    background: #eee;
    border-radius: 5px;
    padding: 5px;
    transition: all ease 0.1s;
    width: 50%;
    margin: 5px;
  }
  .list-item:hover {
    background: #ddd;
    transition: all ease 0.1s;
  }
  </style>
  <p> Below are the folders obtained from your Google Drive. </p>
  <b> Choose a folder to store the model results. </b>
  <hr>
  <div> [!!!REPLACE!!!] </div>

  <script>
    var changestate;
    var selection = new Promise(resolve => {
      changestate = function(i) {
        var el = document.getElementById(i)
        el.style.backgroundColor = '#b0ceff';
        el.style.fontWeight = "900";
        resolve(i);
      }
    });
  </script>
  """
  i = 0
  for root, directories, filenames in os.walk("/content/drive/My Drive"):
    for directory in directories:
      # change the extensions to the type of file we are looking for
        FILE_HTML += """<div id="{}" class="list-item" onclick="changestate('{}')">{}</div>""".format(i, i, directory)
        i += 1
        dirs.append(os.path.join(root,directory))
  display(HTML(JS.replace("[!!!REPLACE!!!]", FILE_HTML)))
  selection = eval_js("selection")
  return dirs[int(selection)]

def google_drive():
  from google.colab import drive
  display(HTML("<i>Please wait while we fetch your Google Drive...</i>"))
  drive.mount('/content/drive')
  clear_output()
  return drive_file_chooser()

def nothing():
  return None

def download():
  return "download"

dropdown_menu({
    "Don't save results": nothing,
    # "Save to Google Drive": google_drive,
    "Download results": download,
}, on_finished)


In [ ]:
#@title #### **Run the demo and save results**
#@markdown Specify the face(s) you hope to blur. The faces will labeled by their position (0, 1, ..., face_number0-1 from left to right). <br>

face_blur = '0,2' #@param {type:"string"}

face_blur = face_blur.split(',')
face_blur = [int(a) for a in face_blur]


def anonymize_face_pixelate(image, blocks=3):
	# divide the input image into NxN blocks
	(h, w) = image.shape[:2]
	xSteps = np.linspace(0, w, blocks + 1, dtype="int")
	ySteps = np.linspace(0, h, blocks + 1, dtype="int")
	# loop over the blocks in both the x and y direction
	for i in range(1, len(ySteps)):
		for j in range(1, len(xSteps)):
			# compute the starting and ending (x, y)-coordinates
			# for the current block
			startX = xSteps[j - 1]
			startY = ySteps[i - 1]
			endX = xSteps[j]
			endY = ySteps[i]
			# extract the ROI using NumPy array slicing, compute the
			# mean of the ROI, and then draw a rectangle with the
			# mean RGB values over the ROI in the original image
			roi = image[startY:endY, startX:endX]
			(B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
			cv2.rectangle(image, (startX, startY), (endX, endY),
				(B, G, R), -1)
	# return the pixelated blurred image
	return image

app = FaceAnalysis(allowed_modules=['detection'])
app.prepare(ctx_id=0, det_size=(640, 640))
swapper = insightface.model_zoo.get_model('inswapper_128.onnx', download=True, download_zip=True)
# img = ins_get_image('t1')
img = cv2.imread(input_file)
faces = app.get(img)
faces = sorted(faces, key = lambda x : x.bbox[0])
res = img.copy()
for i in face_blur:
  (startX, startY, endX, endY) = faces[i]['bbox'].astype("int")
  face_temp = res[startY:endY, startX:endX]
  face_temp = anonymize_face_pixelate(face_temp)
  res[startY:endY, startX:endX] = face_temp

cv2.imwrite("/content/blur.jpg", res)

display(Image('blur.jpg'))

if result_output == "download":
  files.download('blur.jpg')

## 📄 **Documentation**
<hr>
https://github.com/deepinsight/insightface

https://github.com/deepinsight/insightface/tree/master/examples/in_swapper

https://pyimagesearch.com/2020/04/06/blur-and-anonymize-faces-with-opencv-and-python/#pyis-cta-modal
